In [1]:
from sniprq import StemGroups, QuadPaths
import numpy as np
from IPython.display import clear_output
from ipyparallel import Client

DVIEW = Client()[:]
DVIEW.block=True
len(DVIEW)

30

In [2]:
def distribute_job(stem_group):
    import numpy as np
    import networkx as nx
    
    class QuadPaths(object):

        def __init__(self, stems, max_loop_len, loop_probs, bulge_probs):

            self.stems = stems
            self.maxLoopLen = max_loop_len
            self.loopProbs = loop_probs
            self.bulgeProbs = bulge_probs
            self.graphStart = self.stems[0][0]
            self.nodeCodes = {}
            self.nodeQuadCounts = {}
            self.sortedNodeCodes = []
            self.G = self._populate_graph()

        def _populate_graph(self):
            """
                Construct a directed acyclic graph of stems. Each edge
                represents the loop of a putative G4. Is assued that the
                stems are 2D array of shape nx3, where n is the number
                of stems and 3 represents the position of each G in the
                corresponding stem. The array should be pre-sorted by first
                stem position to ensure that the resultant graph is acyclic.
            """
            G = nx.DiGraph()
            all_starts = self.stems[:, 0]
            for stem in self.stems:
                targets = np.where((all_starts > stem[2] + 1) &
                         (all_starts <= stem[2] + self.maxLoopLen + 1))[0]
                n1 = self._stem_encoder(stem)
                self.nodeQuadCounts[n1] = 0
                self.nodeCodes[n1] = stem
                self.sortedNodeCodes.append(n1)
                for t in targets:
                    n2 = self._stem_encoder(self.stems[t])
                    G.add_edge(n1, n2)
            return G

        def _stem_encoder(self, stem_array):
            """
                Represents the bases of a stem in a string notation.
                For example: '41*---*' represents a stem with three
                bulges and stems at position 41, 42 and 46.
            """
            diff = np.diff(stem_array)
            if diff[0] != 1:
                return '%d%s**' % (stem_array[0] - self.graphStart,
                                   '-' * (diff[0] - 1))
            else:
                return '%d*%s*' % (stem_array[0] - self.graphStart,
                                   '-' * (diff[1] - 1))

        def _score_paths(self, qp):
            loop_lens = qp[:, 0][1:] - qp[: , 2][:-1] - 1
            try:
                loop_scores = self.loopProbs[loop_lens - 1] * 1000
            except:
                print (qp)
                return 0

            bulge_lens = [x[2] - x[0] - 2 for x in qp]
            bulge_scores = []
            for i in range(3):
                bulge_scores.append((
                    self.bulgeProbs[bulge_lens[i]] +
                    self.bulgeProbs[bulge_lens[i + 1]]
                )/2)
            scores = bulge_scores*loop_scores
            return int(3/np.sum(1/scores)) + 1

        def generate_quadpaths(self):
            """
                Recursive iteration over all 4-node paths in the
                graph to save all the quad-paths in the graph. This
                function also helps identify stems which do not form
                a quad-path.
            """
            best_triplet = None # caching
            max_l = self.maxLoopLen + 1
            for n1 in self.sortedNodeCodes:
                if n1 not in self.G:
                    continue
                n1_a = self.nodeCodes[n1]
                if best_triplet is not None:
                    if max_l >= best_triplet[0][0] - n1_a[2] > 1:
                        self.nodeQuadCounts[n1] = 1
                        qp = np.vstack([n1_a, best_triplet])
                        score = self._score_paths(qp)
                        yield qp.flatten(), score                    
                        continue
                triplet_score = 0
                best_triplet = None
                s2 = self.G.successors(n1)
                for n2 in self.sortedNodeCodes:
                    if n2 not in s2:
                        continue
                    n2_a = self.nodeCodes[n2]
                    s3 = self.G.successors(n2)
                    for n3 in self.sortedNodeCodes:
                        if n3 not in s3:
                            continue
                        n3_a = self.nodeCodes[n3]
                        s4 = self.G.successors(n3)
                        for n4 in self.sortedNodeCodes:
                            if n4 not in s4:
                                continue
                            n4_a = self.nodeCodes[n4]
                            self.nodeQuadCounts[n1] = 1
                            self.nodeQuadCounts[n2] = 1
                            self.nodeQuadCounts[n3] = 1
                            self.nodeQuadCounts[n4] = 1
                            qp = np.array([n1_a, n2_a, n3_a, n4_a])
                            score = self._score_paths(qp)
                            if score > triplet_score:
                                best_triplet = qp[1:]
                                triplet_score = score
                            yield qp.flatten(), score
                            # # Makes searches non-exhaustive but
                            # # gives a great performance boost
                            if n4_a[2] - n4_a[0] == 2:
                                break
            self._remove_non_quad_nodes()

        def _remove_non_quad_nodes(self):
            """
                Remove nodes that do not occur in any quad-path from
                the graph.
            """
            del_nodes = []
            for node in self.G.nodes():
                if self.nodeQuadCounts[node] == 0:
                    del_nodes.append(node)
            for node in del_nodes:
                self.G.remove_node(node)
            return True

    loop_probs = np.array([
    0.50860458,  0.15217531,  0.08495005,  0.0549146 ,  0.03100226,
    0.03100226,  0.02320335,  0.01585562,  0.01160168,  0.00986142,
    0.00928134,  0.00689655,  0.00702546,  0.00670319,  0.00496294,
    0.00489849,  0.00315823,  0.0026426 ,  0.00270706,  0.00238479,
    0.00283597,  0.00174025,  0.00186916,  0.00212697,  0.00199807, 0.00174025])
    loop_probs = loop_probs/loop_probs[0]
    bulge_probs = np.array([ 0.78772061,  0.13447676,  0.05100671,  0.02679592])
    bulge_probs = bulge_probs/bulge_probs[0]
    
    qp_object = QuadPaths(stem_group, 26, loop_probs, bulge_probs)
    qps, graph_info = {}, []
    n = 0
    for qp, s in qp_object.generate_quadpaths():
        n += 1
        for i in qp:
            if i not in qps:
                qps[i] = s
            elif s > qps[i]:
                qps[i] = s
    if len(qp_object.G.nodes()) > 0:
        nodes = np.array(list(qp_object.nodeCodes.values()))
        graph_info = [len(qp_object.G.nodes()), len(qp_object.G.edges()),
                      n, nodes.min(), nodes.max()]
    return qps, graph_info

fasta_loc = "/home/parashar/scratch/hg19_resource/chromosomes"
chroms = ['chr'+str(x) for x in range(1,23)] + ['chrX', 'chrY']
for chrom in chroms:
    for base, strand in zip(['G', 'C'], ['positive', 'negative']):
        print (chrom, base)
        chrom_seq = "".join([x.rstrip('\n').upper() for x in 
                             open('%s/%s.fa' % (fasta_loc, chrom)).readlines()[1:]])
        scores = np.zeros(len(chrom_seq), dtype=int)
        graph_info = []
        sg_object = StemGroups(chrom_seq, 26, 3, base)
        temp = []
        for stem_group in sg_object.generate_stems():
            temp.append(stem_group)
            if len(temp) == 60000:
                res = DVIEW.map_sync(distribute_job, temp)
                for qps, gi in res:
                    idx, s = [], []
                    for k, v in qps.items():
                        idx.append(k)
                        s.append(v)
                    scores[idx] = s
                    if len(gi) == 5:
                        graph_info.append(gi)
                temp = []
        if len(temp) > 0:
            res = DVIEW.map_sync(distribute_job, temp)
            for qps, gi in res:
                idx, s = [], []
                for k, v in qps.items():
                    idx.append(k)
                    s.append(v)
                scores[idx] = s
                if len(gi) == 5:
                    graph_info.append(gi)
        np.save('../data/sniprq_scores/%s_%s' % (chrom, strand), scores)
        np.save('../data/sniprq_scores/%s_%s_info' % (chrom, strand),
                np.array(graph_info))

chr1 G


100%|██████████| 249250616/249250616 [1:14:10<00:00, 56009.67it/s] 


chr1 C


100%|██████████| 249250616/249250616 [1:14:18<00:00, 55908.93it/s] 


chr2 G


...

100%|██████████| 155270555/155270555 [45:07<00:00, 57348.56it/s]  


chrY G


100%|██████████| 59373561/59373561 [15:35<00:00, 63486.48it/s]  


chrY C


100%|██████████| 59373561/59373561 [16:26<00:00, 60198.79it/s]  
